# Company Information Endpoint

This notebook implements a Flask API endpoint to retrieve detailed company information using Yahoo Finance API.

## Features:
- Company name, business summary, industry, sector
- Key officers' names and titles
- Input validation for stock symbols
- Error handling and JSON responses

## Step 1: Install Required Dependencies

In [ ]:
# Run this cell to install required packages
!pip install flask yfinance requests

## Step 2: Import Required Libraries

In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
import json
from datetime import datetime
import re

## Step 3: Initialize Flask Application

In [ ]:
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False

## Step 4: Create Helper Functions

In [ ]:
def validate_symbol(symbol):
    """
    Validate if the provided symbol is a valid format
    """
    if not symbol or len(symbol) < 1 or len(symbol) > 10:
        return False
    return symbol.replace('.', '').replace('-', '').isalnum()

def search_company_symbol(company_name):
    """
    Search for company symbol using company name
    This is a simplified approach - in production, you'd use a proper symbol lookup service
    """
    try:
        # Clean the company name for search
        clean_name = re.sub(r'[^a-zA-Z0-9\s]', '', company_name).strip().upper()

        # Common company name to symbol mappings (you can expand this)
        common_mappings = {
            'APPLE': 'AAPL',
            'APPLE INC': 'AAPL',
            'MICROSOFT': 'MSFT',
            'MICROSOFT CORPORATION': 'MSFT',
            'GOOGLE': 'GOOGL',
            'ALPHABET': 'GOOGL',
            'AMAZON': 'AMZN',
            'TESLA': 'TSLA',
            'META': 'META',
            'FACEBOOK': 'META',
            'NETFLIX': 'NFLX',
            'NVIDIA': 'NVDA',
            'INTEL': 'INTC',
            'IBM': 'IBM',
            'ORACLE': 'ORCL',
            'SALESFORCE': 'CRM',
            'ADOBE': 'ADBE',
            'PAYPAL': 'PYPL',
            'ZOOM': 'ZM',
            'SPOTIFY': 'SPOT'
        }

        # Check direct mapping first
        if clean_name in common_mappings:
            return common_mappings[clean_name]

        # Check partial matches
        for name_key, symbol in common_mappings.items():
            if clean_name in name_key or name_key in clean_name:
                return symbol

        # If no mapping found, try using the input as a symbol directly
        if len(company_name) <= 5 and company_name.isalpha():
            return company_name.upper()

        return None

    except Exception as e:
        print(f"Error searching for symbol: {str(e)}")
        return None

def format_key_officers(officers_data):
    """
    Format officers data into the required structure with Name and Title only
    """
    if not officers_data:
        return []

    key_officers = []
    for officer in officers_data:
        key_officers.append({
            'Name': officer.get('name', 'N/A'),
            'Title': officer.get('title', 'N/A')
        })
    return key_officers

## Step 5: Create Company Information Retrieval Function

In [ ]:
def get_company_information_by_name(company_name):
    """
    Retrieve company information using company name (converts to symbol first)
    Returns data in the exact format specified by user requirements
    """
    try:
        # First, find the symbol for the company name
        symbol = search_company_symbol(company_name)

        if not symbol:
            return {
                'error': f"Could not find symbol for company: {company_name}",
                'suggestion': "Please try entering the stock symbol directly (e.g., AAPL, MSFT)"
            }

        # Create ticker object
        ticker = yf.Ticker(symbol)

        # Get company info
        info = ticker.info

        # Check if valid data was returned
        if not info or len(info) < 5:
            return {
                'error': f"No valid company data found for symbol: {symbol}",
                'tried_symbol': symbol
            }

        # Extract company information in the exact format requested
        company_data = {
            'Symbol': info.get('symbol', symbol.upper()),
            'Company_name': info.get('longName', info.get('shortName', 'N/A')),
            'Business_summary': info.get('longBusinessSummary', 'N/A'),
            'Industry': info.get('industry', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Key_officers': format_key_officers(info.get('companyOfficers', []))
        }

        return company_data

    except Exception as e:
        return {
            'error': f"Error retrieving data: {str(e)}",
            'company_name': company_name
        }

def get_company_information_by_symbol(symbol):
    """
    Retrieve company information directly using symbol
    """
    try:
        # Create ticker object
        ticker = yf.Ticker(symbol.upper())

        # Get company info
        info = ticker.info

        # Check if valid data was returned
        if not info or len(info) < 5:
            return None

        # Extract company information in the exact format requested
        company_data = {
            'Symbol': info.get('symbol', symbol.upper()),
            'Company_name': info.get('longName', info.get('shortName', 'N/A')),
            'Business_summary': info.get('longBusinessSummary', 'N/A'),
            'Industry': info.get('industry', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Key_officers': format_key_officers(info.get('companyOfficers', []))
        }

        return company_data

    except Exception as e:
        print(f"Error retrieving data for {symbol}: {str(e)}")
        return None

## Step 6: Create User Input Function

In [ ]:
def get_company_info_interactive():
    """
    Interactive function that asks user for company name and retrieves information
    """
    print("=" * 50)
    print("Company Information Retrieval System")
    print("=" * 50)

    # Ask user for company name
    company_name = input("Enter the company name: ").strip()

    if not company_name:
        print("Error: Company name cannot be empty.")
        return None

    print(f"\nSearching for information about: {company_name}")
    print("-" * 40)

    # Get company information
    company_data = get_company_information_by_name(company_name)

    # Check if there was an error
    if 'error' in company_data:
        print(f"Error: {company_data['error']}")
        if 'suggestion' in company_data:
            print(f"Suggestion: {company_data['suggestion']}")
        return None

    # Display the results
    print("\n✅ Company Information Retrieved Successfully!")
    print("=" * 50)

    print(f"Symbol: {company_data['Symbol']}")
    print(f"Company Name: {company_data['Company_name']}")
    print(f"Industry: {company_data['Industry']}")
    print(f"Sector: {company_data['Sector']}")

    print("\nBusiness Summary:")
    print("-" * 20)
    # Truncate business summary for better display
    summary = company_data['Business_summary']
    if len(summary) > 300:
        summary = summary[:300] + "..."
    print(summary)

    print("\nKey Officers:")
    print("-" * 15)
    if company_data['Key_officers']:
        for i, officer in enumerate(company_data['Key_officers'][:5], 1):  # Show top 5 officers
            print(f"{i}. {officer['Name']} - {officer['Title']}")
    else:
        print("No officer information available.")

    return company_data

## Step 7: Create Flask API Endpoints

In [ ]:
@app.route('/api/company/name', methods=['POST'])
def get_company_by_name_endpoint():
    """
    API endpoint to retrieve company information by company name
    """
    try:
        data = request.get_json()

        if not data or 'company_name' not in data:
            return jsonify({
                'error': 'Missing company_name',
                'message': 'Request body must contain a "company_name" field'
            }), 400

        company_name = data['company_name'].strip()

        if not company_name:
            return jsonify({
                'error': 'Empty company name',
                'message': 'Company name cannot be empty'
            }), 400

        # Get company information
        company_data = get_company_information_by_name(company_name)

        # Check if there was an error
        if 'error' in company_data:
            return jsonify(company_data), 404

        return jsonify({
            'success': True,
            'data': company_data
        }), 200

    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'message': str(e)
        }), 500

@app.route('/api/company/<symbol>', methods=['GET'])
def get_company_info_endpoint(symbol):
    """
    API endpoint to retrieve company information by symbol
    """
    try:
        # Validate symbol format
        if not validate_symbol(symbol):
            return jsonify({
                'error': 'Invalid symbol format',
                'message': 'Symbol must be 1-10 characters, alphanumeric with optional dots or hyphens'
            }), 400

        # Get company information
        company_data = get_company_information_by_symbol(symbol)

        if company_data is None:
            return jsonify({
                'error': 'Company not found',
                'message': f'No company information found for symbol: {symbol.upper()}'
            }), 404

        return jsonify({
            'success': True,
            'data': company_data
        }), 200

    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'message': str(e)
        }), 500

## Step 8: Add Health Check and Documentation Endpoints

In [ ]:
@app.route('/health', methods=['GET'])
def health_check():
    """
    Health check endpoint
    """
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'service': 'Company Information API'
    }), 200

@app.route('/api/docs', methods=['GET'])
def api_documentation():
    """
    API documentation endpoint
    """
    docs = {
        'title': 'Company Information API',
        'version': '1.0.0',
        'description': 'Retrieve detailed company information using Yahoo Finance API',
        'endpoints': {
            'GET /api/company/<symbol>': {
                'description': 'Get company information by symbol',
                'parameters': {
                    'symbol': 'Stock symbol (e.g., AAPL, MSFT)'
                },
                'example': '/api/company/AAPL'
            },
            'POST /api/company/name': {
                'description': 'Get company information by company name',
                'body': {
                    'company_name': 'Company name (required)'
                },
                'example': '{"company_name": "Apple Inc"}'
            }
        }
    }
    return jsonify(docs), 200

## Step 9: Interactive Company Information Lookup

In [15]:
# Run this cell to start the interactive company lookup
# This will ask you to "Enter the company name:" and retrieve the information

company_info = get_company_info_interactive()

Company Information Retrieval System
Enter the company name: open ai

Searching for information about: open ai
----------------------------------------
Error: Could not find symbol for company: open ai
Suggestion: Please try entering the stock symbol directly (e.g., AAPL, MSFT)
